In [14]:
import numpy as np

import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data

In [15]:
# 加载了数据
mnist = input_data.read_data_sets('./',one_hot=True)

Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz


In [16]:
#创建变量占位符

X = tf.placeholder(dtype=tf.float64,shape=[None,784])

y = tf.placeholder(dtype=tf.float64,shape=[None,10])

In [25]:
#卷积核    在卷积神经网络中就是变量
#变量生成方法
def gen_v(shape):
    return tf.Variable(initial_value=tf.random_normal(dtype=tf.float64,shape=shape,stddev=0.1),dtype=tf.float64)

In [26]:
#定义方法，完成卷积操作

def conv(input_data,filter_):
    return tf.nn.conv2d(input=input_data,filter=filter_,strides=[1,1,1,1],padding='SAME')


#定义，池化操作
def pool(input_data):
    return tf.nn.max_pool(value=input_data,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')    

In [27]:
# 第一层卷积
input_data1 = tf.reshape(X,shape = [-1,28,28,1])

# 卷积核
filter1 = gen_v(shape = [3,3,1,64])

# 偏差bias
b1 = gen_v(shape = [64])

conv1 = conv(input_data1,filter1) + b1

# 池化
pool1 = pool(conv1)

# 激活函数
active1 = tf.nn.relu(pool1)
active1

Instructions for updating:
Colocations handled automatically by placer.


<tf.Tensor 'Relu:0' shape=(?, 14, 14, 64) dtype=float64>

In [28]:
# 第二层卷积
# 使用的是第一层卷积的数据

# 卷积核
filter2 = gen_v(shape = [3,3,64,64])

# 偏差bias
b2 = gen_v(shape = [64])

conv2 = conv(active1,filter2) + b2

# 池化
pool2 = pool(conv2)

# 激活函数
active2 = tf.nn.sigmoid(pool2)
active2

<tf.Tensor 'Sigmoid:0' shape=(?, 7, 7, 64) dtype=float64>

In [30]:
#全连接层
#1024个连接，1024个方程，1024神经元

fc_w = gen_v(shape = [7*7*64,1024])

fc_b = gen_v(shape = [1024])

conn = tf.matmul(tf.reshape(active2,shape = [-1,7*7*64]),fc_w) + fc_b
conn

<tf.Tensor 'add_3:0' shape=(?, 1024) dtype=float64>

In [31]:
#dropout  防止过拟合
kp = tf.placeholder(dtype=tf.float64)
dropout = tf.nn.dropout(conn,keep_prob=kp)
dropout

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


<tf.Tensor 'dropout/mul:0' shape=(?, 1024) dtype=float64>

In [32]:
#输出层
out_w = gen_v(shape = [1024,10])
out_b = gen_v(shape = [10])

out = tf.matmul(dropout,out_w) + out_b
out

<tf.Tensor 'add_4:0' shape=(?, 10) dtype=float64>

In [33]:
#预测的概率   非真实分布
prob = tf.nn.softmax(out)
prob

<tf.Tensor 'Softmax:0' shape=(?, 10) dtype=float64>

In [34]:
#真实概率
y

<tf.Tensor 'Placeholder_3:0' shape=(?, 10) dtype=float64>

In [35]:
#交叉熵   越小越准确
cost = tf.reduce_mean(tf.reduce_sum(y*tf.log(1/prob),axis = -1))
cost

<tf.Tensor 'Mean:0' shape=() dtype=float64>

In [36]:
#交叉熵   调用方法
cost2 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=prob))
cost2

<tf.Tensor 'Mean_1:0' shape=() dtype=float64>

In [37]:
#AdamOptimizer  梯度下降的优化
adam = tf.train.AdamOptimizer()

In [38]:
optimizer =  adam.minimize(cost)
optimizer

Instructions for updating:
Use tf.cast instead.


<tf.Operation 'Adam' type=NoOp>

In [39]:
config = tf.ConfigProto()

In [41]:
x=3
y=4
z=2
with tf.device('/gpu:0'):
    a=tf.multiply(x,x)
    b=tf.multiply(a,y)
with tf.device('/gpu:1'):
    c=tf.add(y,z)
# 新建 session with log_device_placement 并设置为 True.
sess=tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True))
# 运行这个 op.
d=tf.add(b,c)
print(sess.run(d))

sess.close()

42


In [42]:

with tf.device('/cpu:0'):
    a=tf.multiply(x,x)
    b=tf.multiply(a,y)
    
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True))

print(sess.run(fetches = a))
sess.close()

9


In [43]:
a=tf.multiply(x,x)
b=tf.multiply(a,y)
    
sess = tf.Session()

print(sess.run(fetches = a))
sess.close()

InvalidArgumentError: Cannot assign a device for operation Add_5: node Add_5 (defined at <ipython-input-40-dca02766138a>:8) was explicitly assigned to /device:GPU:1 but available devices are [ /job:localhost/replica:0/task:0/device:CPU:0 ]. Make sure the device specification refers to a valid device. The requested device appears to be a GPU, but CUDA is not enabled.
	 [[node Add_5 (defined at <ipython-input-40-dca02766138a>:8) ]]

Caused by op 'Add_5', defined at:
  File "D:\anacondna\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\anacondna\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\anacondna\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "D:\anacondna\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "D:\anacondna\lib\site-packages\ipykernel\kernelapp.py", line 499, in start
    self.io_loop.start()
  File "D:

In [44]:
saver = tf.train.Saver()

In [45]:
# gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.6)

with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
    
    sess.run(tf.global_variables_initializer())
    
    
    for i in range(10):
        c = 0
        for j in range(100):
            X_train,y_train = mnist.train.next_batch(550)
            
            optimizer_,cost_ = sess.run(fetches = [optimizer,cost],feed_dict= {X:X_train,y:y_train,kp:0.5})
            
            c += cost_/100
            print('里层循环次数：%d，每次损失：%0.4f'%(j,cost_))
            
        print('--------------执行次数：%d损失函数是：%0.4f----------------'%(i,c))
        saver.save(sess,save_path='./cnn/model',global_step=i)

TypeError: Cannot interpret feed_dict key as Tensor: Can not convert a int into a Tensor.

In [48]:
with tf.Session() as sess:
    saver.restore(sess,'./cnn/model-9')
    
    X_test,y_test = mnist.test.next_batch(2000)
    
    
    prob_ = sess.run(prob,feed_dict = {X:X_test,kp:1.0})
    
    
    y_test = y_test.argmax(axis = -1)
    
    prob_ = prob_.argmax(axis = -1)
    
    print((y_test == prob_).mean())

INFO:tensorflow:Restoring parameters from ./cnn/model-9


InvalidArgumentError: Restoring from checkpoint failed. This is most likely due to a mismatch between the current graph and the graph from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

Cannot assign a device for operation Add_5: node Add_5 (defined at <ipython-input-40-dca02766138a>:8) was explicitly assigned to /device:GPU:1 but available devices are [ /job:localhost/replica:0/task:0/device:CPU:0 ]. Make sure the device specification refers to a valid device. The requested device appears to be a GPU, but CUDA is not enabled.
	 [[node Add_5 (defined at <ipython-input-40-dca02766138a>:8) ]]

Caused by op 'Add_5', defined at:
  File "D:\anacondna\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\anacondna\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\anacondna\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  